# Lotka Volterra UPINN

In [1]:
import sys
sys.path.append('../')
import numpy as np
import torch
import torch.nn as nn
from tqdm import tqdm
from utils.NeuralNets import FNN, ScalingLayer
from utils.DataGenerators import LotkaVolterra
from utils.Utils import sample_with_noise

# Plotly
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_dark"

def to_numpy(tensor):
    if type(tensor) != torch.Tensor:
        return tensor
    return tensor.squeeze().detach().cpu().numpy()

Consider the Lotka-Volterra equations, which describe the dynamics of a predator-prey system:

\begin{align}
\frac{dx}{dt} &= \alpha x - \beta x y, \\
\frac{dy}{dt} &= - \delta y + \gamma x y,
\end{align}

where $x$ is the number of prey, $y$ is the number of predators, and $\alpha$, $\beta$, $\gamma$, and $\delta$ are positive constants.

### Problem definition
Using the example from Podina et al. (2023).

In [ ]:
alpha, beta, gamma, delta = 1.3, 0.9, 0.8, 1.8
x0, y0 = 0.44249296, 4.6280594

alpha, beta, gamma, delta = 2/3, 4/3, 1.0, 1.0
x0, y0 = 1.0, 1.0

LV = LotkaVolterra(alpha, beta, gamma, delta, torch.tensor([x0, y0], dtype=torch.float32))

### Generate data

In [3]:
# time_int = [0, 3]
time_int = [0, 20]
N = 1000
t = torch.linspace(time_int[0], time_int[1], N)

X = LV.solve(t)

t_s, X_s = sample_with_noise(30, t, X)

In [37]:
# Visualize
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_f.detach().numpy(), y=x_f, mode='lines', name='x: true', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=t_f.detach().numpy(), y=y_f, mode='lines', name='y: true', line=dict(color='red')))
fig.add_trace(go.Scatter(x=t_sample.detach().numpy(), y=x_sample, mode='markers', name='x: noisy sample', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=t_sample.detach().numpy(), y=y_sample, mode='markers', name='y: noisy sample', line=dict(color='red')))
fig.update_layout(title='Lotka-Volterra Model', xaxis_title='Time', yaxis_title='Population')
fig.show()

### Instantiate the model

In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

f_known = FNN([1, 64, 64, 2])
f_known.to(device)
f_unknown = FNN([2, 16, 16, 2])
f_unknown.to(device)

FNN(
  (scaling): ScalingLayer()
  (layers): ModuleList(
    (0): Linear(in_features=2, out_features=16, bias=True)
    (1): Linear(in_features=16, out_features=16, bias=True)
    (2): Linear(in_features=16, out_features=2, bias=True)
  )
  (act_fn): ReLU()
)

In [6]:
LV.X0

[1.0, 1.0]

### Training loop

In [4]:
import wandb

wandb.init(
    project='Master-Thesis',
    config={
        "learning_rate": 1e-3,
        "Archtechture": "FNN",
        "Problem": "Lotka-Volterra",
        "Epochs": 30000,
        "Optimizer": "Adam",
    }
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: megajosni. Use `wandb login --relogin` to force relogin


In [ ]:
epochs = 30000
lr = 1e-3
optimizer = torch.optim.Adam([*f_known.parameters(), *f_unknown.parameters()], lr=lr)
# optimizer = torch.optim.LBFGS([*f_known.parameters(), *f_unknown.parameters()], lr=1, history_size=10, line_search_fn="strong_wolfe", tolerance_grad=1e-32, tolerance_change=1e-32)


# Weight scaling for the loss function
lambda1, lambda2, lambda3 = 1, 1, 1

progress_bar = tqdm(range(epochs), desc="Training", unit="epoch")

# Move the data to the device and convert to float
t_sample = t_sample.to(device).reshape(-1, 1).float()
x_sample = x_sample.to(device).reshape(-1, 1).float()
y_sample = y_sample.to(device).reshape(-1, 1).float()
t_f = t_f.to(device).reshape(-1, 1).float()

for epoch in progress_bar:
    def closure():
        optimizer.zero_grad()
        
        # Initial condition loss
        x0, y0 = LV.X0
        t0 = torch.tensor([[0.0]], device=device).float()
        X0_pred = f_known(t0)
        ic_loss = nn.MSELoss()(X0_pred, torch.tensor([[x0, y0]], device=device).float())

        # Known dynamics loss
        X_f = f_known(t_f)
        x_f, y_f = X_f[:, 0:1], X_f[:, 1:2]
        dxdt = torch.autograd.grad(x_f, t_f, torch.ones_like(x_f), create_graph=True)[0]
        dydt = torch.autograd.grad(y_f, t_f, torch.ones_like(y_f), create_graph=True)[0]

        res_pred = f_unknown(X_f)
        res_x, res_y = res_pred[:, 0:1], res_pred[:, 1:2]
        dudt = torch.hstack([
            dxdt - LV.alpha * x_f + LV.beta * x_f * y_f - res_x,
            dydt + LV.delta * y_f - res_y
        ])
        pde_loss = torch.mean(dudt[:, 0] ** 2) + torch.mean(dudt[:, 1] ** 2)

        # Data loss
        X_pred = f_known(t_sample)
        data_loss = nn.MSELoss()(X_pred, torch.hstack([x_sample, y_sample]))

        # Total loss
        loss = lambda1 * ic_loss + lambda2 * pde_loss + lambda3 * data_loss
        progress_bar.set_postfix({
            'Loss': loss.item(), 
            'IC Loss': ic_loss.item(), 
            'PDE Loss': pde_loss.item(), 
            'Data Loss': data_loss.item()
        })

        wandb.log({
            "Loss": loss.item(),
            "IC Loss": ic_loss.item(),
            "PDE Loss": pde_loss.item(),
            "Data Loss": data_loss.item()
        })

        loss.backward(retain_graph=True)  # Gradients are calculated here

        return loss
    
    if epoch % 100 == 0:
        # Make plot
        with torch.no_grad():
            t_pred = torch.linspace(0, 20, 1000).reshape(-1, 1).to(device).float()
            X_pred = f_known(t_pred)
            x_pred, y_pred = to_numpy(X_pred[:, 0:1]), to_numpy(X_pred[:, 1:2])
            res_pred = f_unknown(X_pred)
            res_x, res_y = to_numpy(res_pred[:, 0:1]), to_numpy(res_pred[:, 1:2])
            t_pred = to_numpy(t_pred)

            # Create the plotly figure
            fig = go.Figure()
            fig.add_trace(go.Scatter(x=t_pred, y=x_pred, mode='lines', name='x: prediction', line=dict(color='blue')))
            fig.add_trace(go.Scatter(x=t_pred, y=y_pred, mode='lines', name='y: prediction', line=dict(color='red')))
            fig.add_trace(go.Scatter(x=to_numpy(t_sample), y=to_numpy(x_sample), mode='markers', name='x: noisy sample', line=dict(color='blue')))
            fig.add_trace(go.Scatter(x=to_numpy(t_sample), y=to_numpy(y_sample), mode='markers', name='y: noisy sample', line=dict(color='red')))
            fig.update_layout(title='Lotka-Volterra Model', xaxis_title='Time', yaxis_title='Population')

            path_to_plotly_html = "./plotly_figure.html"
            fig.write_html(path_to_plotly_html, auto_play=False)
            table = wandb.Table(columns=["plotly_figure"])
            table.add_data(wandb.Html(path_to_plotly_html))
            wandb.log({"test_table": table})
  

    optimizer.step(closure)  # Pass the closure to LBFGS


# Save the model
torch.save(f_known.state_dict(), 'models/lotka-volterra/f_known1.pt')
torch.save(f_unknown.state_dict(), 'models/lotka-volterra/f_unknown1.pt')

Training:   0%|          | 0/30000 [00:00<?, ?epoch/s]


UnboundLocalError: cannot access local variable 't_sample' where it is not associated with a value

In [ ]:
# Load the model
f_known.load_state_dict(torch.load('models/lotka-volterra/f_known1.pt'))
f_unknown.load_state_dict(torch.load('models/lotka-volterra/f_unknown1.pt'))

C:\Users\jonas\AppData\Local\Temp\ipykernel_18688\3852120688.py:2: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

C:\Users\jonas\AppData\Local\Temp\ipykernel_18688\3852120688

<All keys matched successfully>

In [ ]:
def to_numpy(tensor):
    if type(tensor) != torch.Tensor:
        return tensor
    return tensor.squeeze().detach().cpu().numpy()

In [ ]:
with torch.no_grad():
    t_f = torch.linspace(0, 20, 1000).reshape(-1, 1).to(device).float()
    X_pred = f_known(t_f)
    x_pred, y_pred = to_numpy(X_pred[:, 0:1]), to_numpy(X_pred[:, 1:2])
    res_pred = f_unknown(X_pred)
    res_x, res_y = to_numpy(res_pred[:, 0:1]), to_numpy(res_pred[:, 1:2])
    t_f = to_numpy(t_f)
    t_sample = to_numpy(t_sample)
    x_sample = to_numpy(x_sample)
    y_sample = to_numpy(y_sample)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=t_f, y=x_pred, mode='lines', name='x: prediction', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=t_f, y=y_pred, mode='lines', name='y: prediction', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=t_sample, y=x_sample, mode='markers', name='x: noisy sample', line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=t_sample, y=y_sample, mode='markers', name='y: noisy sample', line=dict(color='red')))
    fig.update_layout(title='Lotka-Volterra Model', xaxis_title='Time', yaxis_title='Population')
    fig.show()

In [ ]:
y_missing = -LV.delta * y_pred + LV.gamma * x_pred * y_pred

# Plot the residuals
fig = go.Figure()
fig.add_trace(go.Scatter(x=t_f, y=res_x, mode='lines', name='G1', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=t_f, y=res_y, mode='lines', name='G2', line=dict(color='red')))
fig.add_trace(go.Scatter(x=t_f, y=y_missing, mode='lines', name='γ*x*y', line=dict(color='green')))
fig.update_layout(title='Residuals', xaxis_title='Time', yaxis_title='Residual')
fig.show()

# Recover missing term of the Lotka-Volterra equations with PySINDy

In [ ]:
# PySINDy
import pysindy as ps

ModuleNotFoundError: No module named 'pysindy'

In [ ]:
# Get relevant data
X = to_numpy(X_pred)
dX = to_numpy(res_pred)
t = t_f

# Define and fit PySINDy models for each residual
sindy_model = ps.SINDy(feature_library=ps.PolynomialLibrary(degree=2), optimizer=ps.STLSQ(threshold=0.2), feature_names=['x', 'y'])

# Fit models for the unknown dynamics
sindy_model.fit(x=X, x_dot=dX, t=t)

# Print the model
sindy_model.print()


(x)' = 0.000
(y)' = -0.549 1 + 0.727 x + 0.029 y + -0.339 x^2 + 1.040 x y + -0.942 y^2


C:\Users\jonas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\pysindy\optimizers\stlsq.py:201: UserWarning:

Sparsity parameter is too big (0.2) and eliminated all coefficients

